In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
import pandas as pd
import sys,os
import random
import copy

import matplotlib.pyplot as plt
import seaborn as sns

from utils.eval import find_best_matches, generate_exprs

from methods import NMF, PCA, sparse_PCA, moCluster, MOFA2

from methods.utils import interpret_results

# let methods find different numbers of clusters
CLUSTER_RANGE = range(4, 11)

scenarios = ['A', 'B', 'C']
bicluster_sizes = [5, 50, 500]


## Load Gloabl Variables

In [2]:
N = 50
scenario = 'A'
# expression file
exprs_file = f"/local/DESMOND2_data_simulated/simulated/{scenario}/{scenario}.n_genes={N},m=4,std=1,overlap=no.exprs_z.tsv"

# read ground truth from file
ground_truth_file = f"/local/DESMOND2_data_simulated/simulated/{scenario}/{scenario}.n_genes={N},m=4,std=1,overlap=no.biclusters.tsv"
ground_truth = pd.read_csv(ground_truth_file,sep ="\t",index_col=0)
ground_truth["samples"] = ground_truth["samples"].apply(lambda x: set(x.split(" ")))
if "genes" in ground_truth.columns.values:
    ground_truth["genes"] = ground_truth["genes"].apply(lambda x: set(x.split(" ")))

# prepare a dict with sample groups corresponding to known bicluster
known_groups = {}
for group in ground_truth.index.values:
    known_groups[group] = ground_truth.loc[group,"samples"]

exprs = pd.read_csv(exprs_file,sep = "\t",index_col=0)
SAMPLE_SIZES = len(exprs.columns)


def test_method(method, args, known_groups):
    result, runtime = method(**args)
    try:
        best_matches = find_best_matches(result,known_groups, SAMPLE_SIZES, FDR=0.05)
        score = best_matches["J_weighted"].sum()
    except ZeroDivisionError:
        score = 0
    return score, result, runtime


# collect all results from the methods
evaluations = {}
evaluations_all = {}

## Run NMF (not transposed, samples are in H)


In [3]:
exprs_NMF = NMF.preprocess_exprs(exprs)

scored_results = []
for m in range(5):
    for k in CLUSTER_RANGE:
        print('Running NMF with k:', k, '...')
        args = {'exprs':exprs_NMF, 'k':k, 'random_state':m, 'transposed':False}
        test_result = test_method(NMF.run, args, known_groups)
        scored_results.append((k, *test_result))
        evaluations_all[f'NMF_H_randomState={m}_k={k}'] = (k, *test_result)
    
best_result = sorted(scored_results, key = lambda y: y[1], reverse=True)[0]
evaluations[f'NMF_H'] = best_result

Running NMF with k: 4 ...
Running NMF with k: 4 ...
Running NMF with k: 4 ...
Running NMF with k: 4 ...
Running NMF with k: 4 ...


## Run NMF (transposed, samples are in W)

In [4]:
exprs_NMF = NMF.preprocess_exprs(exprs)

scored_results = []
for m in range(5):
    for k in CLUSTER_RANGE:
        print('Running NMF with k:', k, '...')
        args = {'exprs':exprs_NMF.T, 'k':k, 'random_state':m, 'transposed':True}
        test_result = test_method(NMF.run, args, known_groups)
        scored_results.append((k, *test_result))
        evaluations_all[f'NMF_W_randomState={m}_k={k}'] = (k, *test_result)

best_result = sorted(scored_results, key = lambda y: y[1], reverse=True)[0]
evaluations[f'NMF_W'] = best_result

Running NMF with k: 4 ...
Running NMF with k: 4 ...
Running NMF with k: 4 ...
Running NMF with k: 4 ...
Running NMF with k: 4 ...


## Run PCA

In [5]:
scored_results = []
for n in CLUSTER_RANGE:
    print('Running PCA with n components:', n, '...')
    args = {'exprs':exprs.T, 'n':n}
    test_result = test_method(PCA.run, args, known_groups)
    scored_results.append((n, *test_result))
    evaluations_all[f'PCA_{n}'] = (n, *test_result)
    
best_result = sorted(scored_results, key = lambda y: y[1], reverse=True)[0]
evaluations[f'PCA'] = best_result

Running PCA with n components: 4 ...


## Run Sparse PCA

In [6]:
scored_results = []
for n in CLUSTER_RANGE:
    print('Running sparse_PCA with n components:', n, '...')
    args = {'exprs':exprs.T, 'n':n}
    test_result = test_method(sparse_PCA.run, args, known_groups)
    scored_results.append((n, *test_result))
    evaluations_all[f'sparse_PCA_{n}'] = (n, *test_result)
    
best_result = sorted(scored_results, key = lambda y: y[1], reverse=True)[0]
evaluations[f'sparse_PCA'] = best_result

Running sparse_PCA with n components: 4 ...


## Run moCluster

moCluster requires the r package mocluster, usage is described here: https://www.bioconductor.org/packages/release/bioc/vignettes/mogsa/inst/doc/moCluster-knitr.pdf

"""
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("mogsa")
"""

install.packages('data.table')

conda install r-xml is necessary

time is measured in R

In [7]:
scored_results = []
for m in range(1, 11):
    print('m:', m)
    for n in CLUSTER_RANGE:
        print('Running moCluster with n components:', n, '...')
        args = {'exprs_file':exprs_file, 'n_dimensions':m, 'n_cluster':n}
        test_result = test_method(moCluster.run, args, known_groups)
        scored_results.append((n, *test_result))
        evaluations_all[f'moCluster_m={m}_cluster={n}'] = (n, *test_result)

best_result = sorted(scored_results, key = lambda y: y[1], reverse=True)[0]
evaluations[f'moCluster'] = best_result

m: 1
Running moCluster with n components: 4 ...


Loading required package: data.table


      [,1]
[1,] 10000
[2,]   200
calculating component 1 ...
m: 2
Running moCluster with n components: 4 ...


Loading required package: data.table


      [,1]
[1,] 10000
[2,]   200
calculating component 1 ...
calculating component 2 ...
m: 3
Running moCluster with n components: 4 ...


...

      [,1]
[1,] 10000
[2,]   200
calculating component 1 ...
calculating component 2 ...
calculating component 3 ...
calculating component 4 ...
calculating component 5 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
calculating component 6 ...
calculating component 7 ...
calculating component 8 ...
calculating component 9 ...
calculating component 10 ...
calculating component 11 ...
calculating component 12 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
calculating component 13 ...
calculating component 14 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
calculating component 15 ...
calculating component 16 ...
calculating component 17 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
m: 18
Running moCluster with n components: 4 ...


Loading required package: data.table


      [,1]
[1,] 10000
[2,]   200
calculating component 1 ...
calculating component 2 ...
calculating component 3 ...
calculating component 4 ...
calculating component 5 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
calculating component 6 ...
calculating component 7 ...
calculating component 8 ...
calculating component 9 ...
calculating component 10 ...
calculating component 11 ...
calculating component 12 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
calculating component 13 ...
calculating component 14 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
calculating component 15 ...
calculating component 16 ...
calculating component 17 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
calculating component 18 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
m: 19
Running moCluster with n components: 4 ...


Loading required package: data.table


      [,1]
[1,] 10000
[2,]   200
calculating component 1 ...
calculating component 2 ...
calculating component 3 ...
calculating component 4 ...
calculating component 5 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
calculating component 6 ...
calculating component 7 ...
calculating component 8 ...
calculating component 9 ...
calculating component 10 ...
calculating component 11 ...
calculating component 12 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
calculating component 13 ...
calculating component 14 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
calculating component 15 ...
calculating component 16 ...
calculating component 17 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
calculating component 18 ...
  Note: maximum number of iterations was reached, algrithm may not converge.
calculating component 19 ...


## Run MOFA2

Requires R version 4.2

https://www.bioconductor.org/packages/release/bioc/html/MOFA2.html

pip install mofapy2

In [8]:
scored_results = []
for s in range(5):
    print('s:', s)
    for m in range(1, 11):
        print('m:', m)
        for n in CLUSTER_RANGE:
            print('Running MOFA2 with n components:', n, '...')
            args = {'exprs_file':exprs_file, 'n_factors':m, 'n_cluster':n, 'seed':s }
            test_result = test_method(MOFA2.run, args, known_groups)
            scored_results.append((n, *test_result))
            evaluations_all[f'MOFA2_factors={m}_cluster={n}_seed={s}'] = (n, *test_result)
            
best_result = sorted(scored_results, key = lambda y: y[1], reverse=True)[0]
evaluations[f'MOFA2'] = best_result

s: 0
m: 1
Running MOFA2 with n components: 4 ...



Attaching package: ‘MOFA2’

The following object is masked from ‘package:stats’:

    predict



[[1]]
[1] 10000   200



Creating MOFA object from a list of matrices (features as rows, sample as columns)...

View names are not specified in the data, using default: view_1

Checking data options...
Checking training options...
Checking model options...
Connecting to the mofapy2 python package using reticulate (use_basilisk = FALSE)... 
    Please make sure to manually specify the right python binary when loading R with reticulate::use_python(..., force=TRUE) or the right conda environment with reticulate::use_condaenv(..., force=TRUE)
    If you prefer to let us automatically install a conda environment with 'mofapy2' installed using the 'basilisk' package, please use the argument 'use_basilisk = TRUE'

sh: /data/home/bba1401/anaconda3/envs/encore2/lib/libtinfo.so.6: no version information available (required by /lib64/libreadline.so.7)



        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Successfully loaded view='view_1' group='group1' with N=200 samples and D=10000 features...


Model options:
- Automatic Relevance Determination prior on the factors: False
- Automatic Relevance Determination prior on the weights: True
- Spike-and-slab prior on the factors: False
- Spike-and-slab prior on the weights: True
Likelihoods:
- View 0 (view_1): gaussian




###########

...

[[1]]
[1] 10000   200



Creating MOFA object from a list of matrices (features as rows, sample as columns)...

View names are not specified in the data, using default: view_1

Checking data options...
Checking training options...
Checking model options...
Connecting to the mofapy2 python package using reticulate (use_basilisk = FALSE)... 
    Please make sure to manually specify the right python binary when loading R with reticulate::use_python(..., force=TRUE) or the right conda environment with reticulate::use_condaenv(..., force=TRUE)
    If you prefer to let us automatically install a conda environment with 'mofapy2' installed using the 'basilisk' package, please use the argument 'use_basilisk = TRUE'

sh: /data/home/bba1401/anaconda3/envs/encore2/lib/libtinfo.so.6: no version information available (required by /lib64/libreadline.so.7)



        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Successfully loaded view='view_1' group='group1' with N=200 samples and D=10000 features...


Model options:
- Automatic Relevance Determination prior on the factors: False
- Automatic Relevance Determination prior on the weights: True
- Spike-and-slab prior on the factors: False
- Spike-and-slab prior on the weights: True
Likelihoods:
- View 0 (view_1): gaussian




###########

Iteration 152: time=0.57, Factors=19
Iteration 153: time=0.58, Factors=19
Iteration 154: time=0.57, Factors=19
Iteration 155: time=0.57, Factors=19
Iteration 156: time=0.59, ELBO=-2902601.52, deltaELBO=216.697 (0.00082407%), Factors=19
Iteration 157: time=0.57, Factors=19
Iteration 158: time=0.58, Factors=19
Iteration 159: time=0.57, Factors=19
Iteration 160: time=0.58, Factors=19
Iteration 161: time=0.59, ELBO=-2902394.57, deltaELBO=206.948 (0.00078700%), Factors=19
Iteration 162: time=0.57, Factors=19
Iteration 163: time=0.57, Factors=19
Iteration 164: time=0.58, Factors=19
Iteration 165: time=0.58, Factors=19
Iteration 166: time=0.59, ELBO=-2902237.49, deltaELBO=157.078 (0.00059735%), Factors=19
Iteration 167: time=0.57, Factors=19
Iteration 168: time=0.57, Factors=19
Iteration 169: time=0.58, Factors=19
Iteration 170: time=0.58, Factors=19
Iteration 171: time=0.59, ELBO=-2902120.79, deltaELBO=116.706 (0.00044382%), Factors=19
Iteration 172: time=0.58, Factors=19
Iteration 173: time

Warning message:
In run_mofa(MOFAobject, outfile) :
  The latest mofapy2 version is 0.6.4, you are using 0.6.6. Please upgrade with 'pip install mofapy2'


In [9]:
df_evaluations = pd.DataFrame(evaluations).T
df_evaluations.columns = ['#cluster', 'Jaccard', 'cluster', 'time (s)']
df_evaluations = df_evaluations.sort_values(by='Jaccard', ascending=False)
df_evaluations.to_csv(f'evaluation_{scenario}_{N}.csv')
df_evaluations

,#cluster,Jaccard,cluster,time (s)
moCluster,4,0.973684,s...,2.755012
MOFA2,4,0.973684,s...,23.394029
NMF_W,4,0.973545,s...,4.86538
NMF_H,4,0.886574,s...,5.93388
sparse_PCA,4,0.632716,s...,399.014365
PCA,4,0.309402,s...,0.147056


In [12]:
df_evaluations_all = pd.DataFrame(evaluations_all).T
df_evaluations_all.columns = ['#cluster', 'Jaccard', 'cluster', 'time (s)']
df_evaluations_all = df_evaluations_all.sort_values(by='Jaccard', ascending=False)
df_evaluations_all.to_csv(f'evaluation_all_{scenario}_{N}.csv')
df_evaluations_all

NameError: name 'evaluations_all' is not defined